In [1]:
# -*- coding: utf-8 -*-
"""
Jupyter Notebook: Combined Planck 2018 CMB + DESI BAO Analysis (ΛCDM)

This notebook performs a combined cosmological parameter analysis using:
1.  Planck 2018 Cosmic Microwave Background (CMB) data:
    * High-multipole Temperature, E-mode Polarization, and Cross-polarization (TT, TE, EE) from Plik.
    * Low-multipole E-mode Polarization (lowE).
    * CMB Lensing reconstruction.
2.  DESI (Dark Energy Spectroscopic Instrument) Baryon Acoustic Oscillation (BAO) data.

The analysis is performed within the framework of the traditional 6-parameter ΛCDM model,
using the Cobaya Bayesian analysis code and GetDist for results visualization.
"""

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path # For more robust path handling

# Ensure matplotlib plots are inline
%matplotlib inline

# Import Cobaya for Bayesian analysis
from cobaya.model import Model
from cobaya.yaml import yaml_load_file
from cobaya.run import run

# Import GetDist for plotting and analysis
# Corrected: Import load_samples directly from getdist
import getdist
from getdist import plots, MCSamples, load_samples 

# --- 1. Define Paths to Data and Likelihoods ---
# IMPORTANT: This path is CRITICAL for Planck data.
# It should point to the directory containing the 'clik' data folders (e.g., Planck_highl_plik_R3.01, Commander_dx12_ell_min2_max29).
# This is typically located in your user's home directory under '.cobaya/data/'.
# Please verify this path on your system. If it's different, update it manually.
cobaya_data_path = str(Path.home() / ".cobaya" / "data") # This is the most common default Cobaya data path

print(f"Assuming Cobaya data is located at: {cobaya_data_path}")

# Paths to Cobaya's core components (likely within your .venv)
# These paths are usually correct if Cobaya itself is installed in the virtual environment.
cobaya_base_path = os.path.join(os.getcwd(), '_packages', 'Lib', 'site-packages', 'cobaya')
likelihoods_path = os.path.join(cobaya_base_path, 'likelihoods')
theories_path = os.path.join(cobaya_base_path, 'theories')

# Path to the Planck 2018 chains output directory
planck_chains_output_dir = 'chains/planck_desi_lcdm_combined/'
os.makedirs(planck_chains_output_dir, exist_ok=True) # Create output directory if it doesn't exist

# --- File Existence Check ---
print("\n--- Verifying Required Data Files ---")

# Check Planck 2018 High-l Plik data
# The 'data_path' parameter within the likelihood config is crucial for Cobaya to find the .clik_binary files.
# The expected structure is cobaya_data_path / 'planck_2018_highl_plik' / 'Planck_highl_plik_R3.01'
plik_clik_dir_name = 'planck_2018_highl_plik' # This is the module name in cobaya/likelihoods
plik_data_actual_path = os.path.join(cobaya_data_path, plik_clik_dir_name, 'Planck_highl_plik_R3.01') # Specific clik data folder
plik_exists = os.path.exists(plik_data_actual_path)
print(f"Planck 2018 High-l Plik data directory '{plik_data_actual_path}': {'EXISTS' if plik_exists else 'NOT FOUND'}")
if not plik_exists:
    print("  (This is the actual *data* directory that the likelihood needs. If not found, run 'cobaya-install planck_2018_highl_plik')")

# Check Planck 2018 Low-l EE data
lowl_clik_dir_name = 'planck_2018_lowl' # This is the module name
lowl_data_actual_path = os.path.join(cobaya_data_path, lowl_clik_dir_name, 'Commander_dx12_ell_min2_max29.clik_binary') # Specific clik data file
lowl_exists = os.path.exists(lowl_data_actual_path)
print(f"Planck 2018 Low-l EE data file '{lowl_data_actual_path}': {'EXISTS' if lowl_exists else 'NOT FOUND'}")
if not lowl_exists:
    print("  (This is the actual *data* file that the likelihood needs. If not found, run 'cobaya-install planck_2018_lowl.EE')")


# Check Planck 2018 Lensing data
lensing_clik_dir_name = 'planck_2018_lensing' # This is the module name
lensing_data_actual_path = os.path.join(cobaya_data_path, lensing_clik_dir_name, 'PlanckLensing2018.clik_binary') # Specific clik data file
lensing_exists = os.path.exists(lensing_data_actual_path)
print(f"Planck 2018 Lensing data file '{lensing_data_actual_path}': {'EXISTS' if lensing_exists else 'NOT FOUND'}")
if not lensing_exists:
    print("  (This is the actual *data* file that the likelihood needs. If not found, run 'cobaya-install planck_2018_lensing')")


# Check DESI BAO data (relative to current working directory)
desi_mean_file = os.path.join(os.getcwd(), 'bao_data-master', 'desi_2024_gaussian_bao_ALL_GCcomb_mean.txt')
desi_cov_file = os.path.join(os.getcwd(), 'bao_data-master', 'desi_2024_gaussian_bao_ALL_GCcomb_cov.txt')

desi_mean_exists = os.path.exists(desi_mean_file)
desi_cov_exists = os.path.exists(desi_cov_file)

print(f"DESI BAO mean file '{desi_mean_file}': {'EXISTS' if desi_mean_exists else 'NOT FOUND'}")
print(f"DESI BAO cov file '{desi_cov_file}': {'EXISTS' if desi_cov_exists else 'NOT FOUND'}")
if not (desi_mean_exists and desi_cov_exists):
    print("  (DESI BAO data files not found. Ensure 'bao_data-master' is in your project root and contains these files.)")

print("--- File Verification Complete ---")

# --- 2. Configure the Cosmological Model and Likelihoods for Cobaya ---

info = {
    'params': {
        'ombh2': {'prior': {'min': 0.019, 'max': 0.026}, 'ref': 0.0224, 'proposal': 0.00005, 'latex': r'\Omega_{\rm b}h^2'},
        'omch2': {'prior': {'min': 0.09, 'max': 0.15}, 'ref': 0.120, 'proposal': 0.0005, 'latex': r'\Omega_{\rm c}h^2'},
        'H0': {'prior': {'min': 40, 'max': 90}, 'ref': 67.4, 'proposal': 0.5, 'latex': r'H_0'},
        'ns': {'prior': {'min': 0.92, 'max': 1.0}, 'ref': 0.965, 'proposal': 0.002, 'latex': 'n_s'},
        'tau': {'prior': {'min': 0.03, 'max': 0.1}, 'ref': 0.054, 'proposal': 0.005, 'latex': r'\tau'},
        'logA': {'prior': {'min': 2.5, 'max': 3.5}, 'ref': 3.044, 'proposal': 0.002, 'latex': r'\log(10^{10}A_s)'},
        'As': {'value': 'lambda logA: 10**(logA)/1e10', 'latex': r'A_s'},
        'sigma8': {'latex': r'\sigma_8'},
        'omegam': {'latex': r'\Omega_m'},
    },

    'theory': {
        'camb': {
            'stop_at_error': False,
            'path': os.path.join(theories_path, 'camb'),
        }
    },

    'likelihood': {
        # Planck 2018 High-l (TT, TE, EE) Plik likelihood
        # 'path' points to the Python wrapper. 'class' points to the class *within* that wrapper module.
        # 'data_path' points to the actual clik data binaries.
        'planck_2018_highl_plik': { 
            'path': os.path.join(likelihoods_path, 'planck_2018_highl_plik'),
            'class': 'planck_2018_highl_plik.TTTEEE', # Explicit Python path to the class
            'l_max_cached': 2508,
            'data_path': plik_data_actual_path, # Use the path derived from cobaya_data_path
        },
        # Planck 2018 Low-l (EE) likelihood
        'planck_2018_lowl_EE': { 
            'path': os.path.join(likelihoods_path, 'planck_2018_lowl'),
            'class': 'planck_2018_lowl.EE', # Explicit Python path to the class
            'data_file': lowl_data_actual_path, # Use the path derived from cobaya_data_path
        },
        # Planck 2018 Lensing likelihood
        'planck_2018_lensing': {
            'path': os.path.join(likelihoods_path, 'planck_2018_lensing'),
            'class': 'planck_2018_lensing.clik', # Explicit Python path to the class
            'data_path': lensing_data_actual_path, # Use the path derived from cobaya_data_path
        },
        # DESI BAO 2024 combined (ALL_GCcomb)
        'desi_2024_bao_all': {
            'path': os.path.join(likelihoods_path, 'bao'),
            'data_file': desi_mean_file, # Use the path derived from os.getcwd()
            'cov_file': desi_cov_file,   # Use the path derived from os.getcwd()
        },
    },

    'sampler': {
        'mcmc': {
            'Rminus1_stop': 0.01,
            'max_tries': 10000,
            'learn_proposal': True,
            'output_every': '60s',
            'measure_speeds': True,
            'oversample_power': 0.4,
            'proposal_scale': 2,
        }
    },
    
    'output': planck_chains_output_dir + 'planck_desi_lcdm_output',
    'debug': False
}

# --- 3. Run the Cobaya Sampler ---
# This part will only run if the file checks above don't raise any issues.
try:
    print("\nStarting Cobaya run for combined Planck + DESI ΛCDM analysis...")
    updated_info, products = run(info)
    print("Cobaya run finished.")
except Exception as e:
    print(f"\nError during Cobaya run: {e}")
    print("Please resolve the file/path issues reported above before re-running.")
    # Exit or prevent further execution if the run failed
    # In a notebook, 'raise' would stop execution. 'exit()' might stop the kernel.
    # For now, we'll let it print the error and continue to the GetDist part
    # but be aware that GetDist might fail if no chains were generated.


# --- 4. Analyze and Plot Results using GetDist ---
print("\nAnalyzing results with GetDist...")

g = plots.get_subplot_plotter(width_inch=10)
g.settings.title_limit = 1
g.settings.alpha_filled_add = 0.8

try:
    # Load the generated chains
    samples = load_samples(planck_chains_output_dir + 'planck_desi_lcdm_output')

    plot_params = ['ombh2', 'omch2', 'H0', 'ns', 'tau', 'sigma8']

    g.triangle_plot(samples, plot_params, filled=True, legend_loc='upper right')
    g.export(planck_chains_output_dir + 'planck_desi_lcdm_triangle.png')
    print(f"Triangle plot saved to: {planck_chains_output_dir}planck_desi_lcdm_triangle.png")

    print("\nMarginalized 1D Statistics:")
    for p in plot_params:
        mean = samples.getMargeStats().pars[samples.index[p]].mean
        std = samples.getMargeStats().pars[samples.index[p]].err
        print(f"{p}: {mean:.4f} +/- {std:.4f}")

    h0_limits = samples.getMargeStats().get_param_conf_limits('H0', alpha=0.32)
    print(f"\nH0 (68% CL): {h0_limits.lower:.2f}, {h0_limits.upper:.2f} km/s/Mpc")

except Exception as e:
    print(f"Error loading or plotting chains: {e}")
    print("Please check if the Cobaya run completed successfully and generated chain files.")

print("\nAnalysis complete. Check the 'chains/planck_desi_lcdm_combined/' directory for output files.")


ImportError: cannot import name 'load_samples' from 'getdist' (C:\Users\GuilhermedeSouzaFern\anaconda3\Lib\site-packages\getdist\__init__.py)

In [3]:
# -*- coding: utf-8 -*-
"""
Jupyter Notebook: Simplified Planck 2018 High-l CMB Analysis (ΛCDM)

This notebook performs a basic cosmological parameter analysis using:
1.  Planck 2018 High-multipole Temperature, E-mode Polarization, and Cross-polarization (TT, TE, EE) from Plik.

The analysis is performed within the framework of the traditional 6-parameter ΛCDM model,
using the Cobaya Bayesian analysis code and GetDist for results visualization.
"""

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path # For more robust path handling

# Ensure matplotlib plots are inline
%matplotlib inline

# Import Cobaya for Bayesian analysis
from cobaya.model import Model
from cobaya.yaml import yaml_load_file
from cobaya.run import run

# Import GetDist for plotting and analysis
import getdist
from getdist import plots, MCSamples, load_samples 

# --- 1. Define Paths to Data and Likelihoods ---
# IMPORTANT: This path is CRITICAL for Planck data.
# It should point to the directory containing the 'clik' data folders
# (e.g., Planck_highl_plik_R3.01, Commander_dx12_ell_min2_max29, PlanckLensing2018.clik_binary).
# This is typically located in your user's home directory under '.cobaya/data/'.
#
# BASED ON YOUR LAST ERROR, YOU LIKELY NEED TO USE THIS PATH:
cobaya_data_path = str(Path.home() / ".cobaya" / "data") 
print(f"Assuming Cobaya data is located at: {cobaya_data_path}")


# Paths to Cobaya's core components (likely within your .venv)
cobaya_base_path = os.path.join(os.getcwd(), '_packages', 'Lib', 'site-packages', 'cobaya')
likelihoods_path = os.path.join(cobaya_base_path, 'likelihoods')
theories_path = os.path.join(cobaya_base_path, 'theories')

# Path to the chains output directory
planck_chains_output_dir = 'chains/planck_highl_lcdm_simplified/'
os.makedirs(planck_chains_output_dir, exist_ok=True) # Create output directory if it doesn't exist

# --- File Existence Check ---
print("\n--- Verifying Required Data Files ---")

# Check Planck 2018 High-l Plik data
plik_clik_dir_name = 'planck_2018_highl_plik' 
# The specific folder name containing the .clik_binary files for plik is usually 'Planck_highl_plik_R3.01'
plik_data_actual_path = os.path.join(cobaya_data_path, plik_clik_dir_name, 'Planck_highl_plik_R3.01') 
plik_exists = os.path.exists(plik_data_actual_path)
print(f"Planck 2018 High-l Plik data directory '{plik_data_actual_path}': {'EXISTS' if plik_exists else 'NOT FOUND'}")
if not plik_exists:
    print("  (CRITICAL: This directory must exist and contain the Planck high-l data.)")
    print(f"  If not found, try running in your terminal/Anaconda Prompt: 'cobaya-install {plik_clik_dir_name}'")
    print(f"  OR manually specify the correct 'cobaya_data_path' in this notebook.")

print("--- File Verification Complete ---")

# --- 2. Configure the Cosmological Model and Likelihoods for Cobaya ---

info = {
    'params': {
        'ombh2': {'prior': {'min': 0.019, 'max': 0.026}, 'ref': 0.0224, 'proposal': 0.00005, 'latex': r'\Omega_{\rm b}h^2'},
        'omch2': {'prior': {'min': 0.09, 'max': 0.15}, 'ref': 0.120, 'proposal': 0.0005, 'latex': r'\Omega_{\rm c}h^2'},
        'H0': {'prior': {'min': 40, 'max': 90}, 'ref': 67.4, 'proposal': 0.5, 'latex': r'H_0'},
        'ns': {'prior': {'min': 0.92, 'max': 1.0}, 'ref': 0.965, 'proposal': 0.002, 'latex': 'n_s'},
        'tau': {'prior': {'min': 0.03, 'max': 0.1}, 'ref': 0.054, 'proposal': 0.005, 'latex': r'\tau'},
        'logA': {'prior': {'min': 2.5, 'max': 3.5}, 'ref': 3.044, 'proposal': 0.002, 'latex': r'\log(10^{10}A_s)'},
        'As': {'value': 'lambda logA: 10**(logA)/1e10', 'latex': r'A_s'},
        'sigma8': {'latex': r'\sigma_8'},
        'omegam': {'latex': r'\Omega_m'},
    },

    'theory': {
        'camb': {
            'stop_at_error': False,
            'path': os.path.join(theories_path, 'camb'),
        }
    },

    'likelihood': {
        # Only Planck 2018 High-l (TT, TE, EE) Plik for now
        'planck_2018_highl_plik': { 
            'path': os.path.join(likelihoods_path, 'planck_2018_highl_plik'),
            'class': 'planck_2018_highl_plik.TTTEEE',
            'l_max_cached': 2508,
            'data_path': plik_data_actual_path, # Use the path derived from cobaya_data_path
        },
    },

    'sampler': {
        'mcmc': {
            'Rminus1_stop': 0.01,
            'max_tries': 10000,
            'learn_proposal': True,
            'output_every': '60s',
            'measure_speeds': True,
            'oversample_power': 0.4,
            'proposal_scale': 2,
        }
    },
    
    'output': planck_chains_output_dir + 'planck_highl_lcdm_output',
    'debug': False
}

# --- 3. Run the Cobaya Sampler ---
# This part will only run if the file checks above don't raise any issues.
run_successful = False
if plik_exists: # Only attempt run if the primary data file is found
    try:
        print("\nStarting Cobaya run for simplified Planck High-l ΛCDM analysis...")
        updated_info, products = run(info)
        print("Cobaya run finished.")
        run_successful = True
    except Exception as e:
        print(f"\nError during Cobaya run: {e}")
        print("Please resolve the file/path issues reported above before re-running.")
else:
    print("\nSkipping Cobaya run due to missing Planck data files.")


# --- 4. Analyze and Plot Results using GetDist ---
print("\nAnalyzing results with GetDist...")

g = plots.get_subplot_plotter(width_inch=10)
g.settings.title_limit = 1
g.settings.alpha_filled_add = 0.8

if run_successful:
    try:
        # Load the generated chains
        samples = load_samples(planck_chains_output_dir + 'planck_highl_lcdm_output')

        plot_params = ['ombh2', 'omch2', 'H0', 'ns', 'tau', 'sigma8']

        g.triangle_plot(samples, plot_params, filled=True, legend_loc='upper right')
        g.export(planck_chains_output_dir + 'planck_highl_lcdm_triangle.png')
        print(f"Triangle plot saved to: {planck_chains_output_dir}planck_highl_lcdm_triangle.png")

        print("\nMarginalized 1D Statistics:")
        for p in plot_params:
            mean = samples.getMargeStats().pars[samples.index[p]].mean
            std = samples.getMargeStats().pars[samples.index[p]].err
            print(f"{p}: {mean:.4f} +/- {std:.4f}")

        h0_limits = samples.getMargeStats().get_param_conf_limits('H0', alpha=0.32)
        print(f"\nH0 (68% CL): {h0_limits.lower:.2f}, {h0_limits.upper:.2f} km/s/Mpc")

    except Exception as e:
        print(f"Error loading or plotting chains: {e}")
        print("Please check if the Cobaya run completed successfully and generated chain files.")
else:
    print("\nSkipping GetDist analysis as Cobaya run was not successful.")

print("\nAnalysis complete. Check the 'chains/planck_highl_lcdm_simplified/' directory for output files (if run was successful).")

ImportError: cannot import name 'load_samples' from 'getdist' (C:\Users\GuilhermedeSouzaFern\anaconda3\Lib\site-packages\getdist\__init__.py)